En primer lugar instalamos y cargamos las librerías necesarias.

In [1]:
import os

from pytube import YouTube #Para descargar los videos
from pydub import AudioSegment
import pydub #Para trabajar con el audio

from pocketsphinx import AudioFile, get_model_path

import re #Para la parte de normalizar
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from autocorrect import Speller #Para los errores ortográficos
spell = Speller(lang='es')
from jiwer import wer #Para la métrica

[nltk_data] Downloading package punkt to /home/jonathan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jonathan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Una vez tenemos todo lo que necesitamos, comenzamos descargando los vídeos, escribiendo los subtítulos y quedándonos con los audios.

In [2]:
videos = []
videos.append({'video': YouTube('https://www.youtube.com/watch?v=UDSGuyxecZo'), 'texto': 'Defendamos esa bandera pionera contra la violencia de género. Pondremos en riesgo espacios de libertad y de igualdad alcanzados con mucho esfuerzo. El Gobierno de La Rioja ya lo ha hecho creando esta dirección general de igualdad que represento. Con su creación no sólo ofrecemos un mensaje claro sobre la esencia que internará toda la acción del gobierno, sino damos un evidente empuje a medidas dirigidas a acabar con la discriminación hacia las mujeres y a eliminar la violencia de género. Nuestra presencia en puestos de responsabilidad en tribunas como la que hoy utilizo para dirigirme a vosotros y a vosotras ya no son excepción. Avanzamos las mujeres, avanzáis los hombres, avanzamos como sociedad. En La Rioja ya son diez años los que llevamos sin lamentar víctimas mortales por violencia de género. Sí, hay razones para el optimismo, pero todavía nos queda mucho por hacer.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=xoASiaGDo08&t=1s'), 'texto': 'Estamos hoy en las obras de esta estación depuradora de Galilea, que, con una inversión de 1,4 millones de euros por parte del consorcio del Gobierno de La Rioja, va también a coger las aguas residuales de otros dos municipios de El Redal y de Corera. Es una obra que ejemplifica muy bien dos de las prioridades del Gobierno de La Rioja como es la atención a todos nuestros municipios, independientemente de su tamaño, para darle los mismos servicios que tiene cualquier ciudadano en cualquier parte de La Rioja, luchando así también contra contra el reto demográfico. Y luego, pues la prioridad del medio ambiente, ¿no? Poner en valor los recursos tan escasos como es el agua y esa necesidad de depurar y devolverlo al medio ambiente pues con calidad para beneficiar a los ecosistemas y también a la salud humana, a la economía y a la sociedad. Así, conseguimos poco a poco pues estos estos distintivos que hemos recibido, este reconocimiento que hemos recibido del ministerio para transición energética, el reto demográfico que nos ha reconocido como la comunidad que que mejor depura el agua en conjunto de España.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=A7oCCguBJHY'), 'texto': 'En La Rioja estamos acabando la tercera campaña masiva de pruebas COVID a todos y cada uno de los residentes. 2.648 pruebas en los 32 centros para personas mayores en La Rioja. Y si ahora acabamos la tercera campaña es porque antes hubo dos: una en abril y otra en junio. Hoy permítanme que les anuncie que vamos a realizar un cribado masivo de COVID-19 entre los 1986 trabajadores de las residencias. Se trata de una ronda extraordinaria de pruebas PCR que tomaremos de manera excepcional para complementar el importantísimo esfuerzo que ya están realizando los centros con sus equipos profesionales. Esta… este cribado, esta ronda extraordinaria empezará, comenzará en los próximos días.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=6yjeUog8X2U'), 'texto': 'Un absoluto convencimiento el trabajo honesto de la inmensa mayoría de la comunidad universitaria riojana y española. El papel de la Universidad de La Rioja va mucho más allá para este gobierno. Durante los próximos años será el pilar fundamental desde el que pivotar el crecimiento equitativo económico y el cambio de modelo productivo de esta región.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=AObE1pT_KQo'), 'texto': 'Agradecer. Agradecer a los más de 12 mil socios que tiene Cruz Roja en La Rioja. A esos más de 3.000 voluntarios en las ocho asambleas que tiene La Rioja y, sobre todo, agradecer la labor que hacen porque están en todo: en un rescate, allí esta Cruz Roja; que, si hay que atender a alguien desfavorecido, ahí está Cruz Roja, ¿no? Y, además, sobre todo la buena imagen que tiene Cruz Roja y la tranquilidad que da a todos los ciudadanos en el momento que ven el emblema de Cruz Roja.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=E_P06RodKxc'), 'texto': 'La educación de calidad tiene que ser una educación pública por encima de todo. Una educación pública de calidad, sostenible decían, en igualdad, equitativa, solidaria. Esta educación es la que va a hacer a la sociedad mucho más firme, más fuerte y es también por asegurarnos el futuro. El alumno, la persona, va a ser el centro de nuestra atención, va a ser el centro y el modelo del nuevo proceso educativo que queremos rediseñar, y va a ser también la guía de nuestra acción para el cambio.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=n_cR-mqd9JY'), 'texto': 'Para nosotros es realmente importante la donación durante todos los meses porque es un acto de generosidad, de humanidad, de solidaridad y porque en navidad estos estos sentimientos quizás se despierten de una manera más importante. Yo sé que los riojanos y las riojanas son generosas siempre y cuando hay una llamada para la donación hay hay una buena buena respuesta.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=TNQMnPyFQbo'), 'texto': 'Los comités empezaron a funcionar a finales de junio, julio y bueno, pues, la primera labor ha sido ponernos un poco de acuerdo en que tenemos un objetivo común y lo siguiente pues ha sido un poco avanzar en lo que cada uno tenía en su mano, ¿no? Los pequeños ayuntamientos han intentado hacer campañas permanentes más allá de las que hacían la consejería de salud o el ministerio, concienciando sobre todo a la gente joven. Y bueno, pues quizá la otra cara es la del trabajo que ha supuesto para para los pequeños ayuntamientos el mantener activa la vida en el medio rural, en los pueblos. Ha sido una época en la que la población se ha disparado y, sin embargo, pues ha habido muchos ayuntamientos que han querido que se mantuviera la vida y los servicios típicos de cada verano a costa de mucho más trabajo y de mucha más inversión.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=vlxpgLeRPEE'), 'texto': 'En el caso particular de la zona básica de salud de Nájera se ha decidido dividir el territorio en dos zonas. En el caso de la zona de la sierra las labores de concienciación fundamentalmente ligadas a las segundas residencias han sido por el momento las acciones más directas. En el caso de la zona de valle las problemáticas se están dirigiendo más, además de lógicamente la común de ocios y confluencia de ciudadanos, a la cuestión agrícola de trabajadores temporales y, más inmediatamente, la cuestión educativa. Esos son los particularidades de idiosincrasias de cada una de estas zonas.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=XygB_x6ajv0'), 'texto': 'Hoy estamos realizando, en el colegio Milenario de la Lengua, la primera entrega de los kits de salud bucodental que se va a hacer extensible a todos los colegios de La Rioja. Digamos que está es la actividad que los niños pueden hacer, pero como digo va en un plan más amplio tanto de salud bucodental como de promoción de los hábitos de vida saludables. Esta consejería quiere estar en los sitios donde sucede todo, donde sucede en la educación, y hacerlo de una forma habitual. Se educa con el ejemplo, entonces los profesores aquí, los padres en casa, en los lugares públicos, ellos, que son esponjas, van a fijarse y van a lo bueno de nosotros. Entonces esa colaboración con la consejería de salud.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=viswBJssw1o'), 'texto': 'Se dice siempre que los niños son el futuro y, realmente, esto es una verdad a medias porque los niños tienen que ser el presente, porque si no somos capaces de defender sus derechos en el presente y en el hoy mal futuro habrá para ellos y para la sociedad.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=g-RpjX9MGJ4'), 'texto': 'Muchísimas gracias por el trabajo que estáis realizando, que habéis realizado y que, por supuesto, vais a seguir realizando. Hoy es un día importante porque hoy es el día mundial de la educación ambiental, así que es el día perfecto para reconocer a todos los alumnos y alumnas, a los profesores, a los educadores, a las educadoras ambientales, directores… bueno, a todos los que lleváis trabajando durante tanto tiempo para que algo que… con lo que yo soñaba cuando estudié, sea una realidad, que es al fin y al cabo concienciar. El trabajo que cada uno de vosotros y cada una de vosotras hacéis, el individual, si se junta con el resto de vuestros compañeros se hace grande, y podemos llegar a conseguir esa protección del medio ambiente, ese futuro sostenible, ese futuro, ese desarrollo en equilibrio que tenemos que tener, que tenemos que ir preparando para las futuras generaciones.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=CNxfT4qkYYo'), 'texto': 'Desde el Gobierno de La Rioja estamos convencidos, como se ha dicho aquí, que una formación de calidad es fundamental para contar con unos cuerpos de policía local profesionales, cercanos, capaces de la respuesta a las necesidades de los ciudadanos y a las necesidades que tienen en cada uno de los municipios dónde estáis. Contamos para ello con un plan de formación continua que cada año tiene una gran participación en sus distintas actividades.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=4xwq6zMHlZA'), 'texto': 'La apuesta de este ejecutivo por la investigación y el compromiso con el papel que ya juega el IER es sigue intacta. El trabajo de investigación, como se comprueba con los éxitos finales, pues se comprueba día a día y la confianza de este gobierno con el Instituto de Estudios Riojanos sigue intacta. Por eso quiero lanzar un mensaje de tranquilidad en este sentido. Este gobierno, con pandemia y sin ella, va a seguir apoyando la promoción, la difusión y divulgación de la ciencia y cultura riojanas y de sus valores.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=uxjgWRTNLn0'), 'texto': 'Tan responsables han sido ellos trabajando y cuidándose como también ha sido responsable toda la población riojana. Por lo tanto, si tuviéramos la suerte y la justicia, diría yo, de poder subir a segunda división, pido responsabilidad en la celebración. Todos y cada uno de nosotros podemos, desde nuestros balcones, ventanas, desde sentaditos en una terraza, con toda la responsabilidad posible, celebrar esa maravillosa noticia. Estamos todos ansiosos de maravillosas noticias, de buenas noticias. Pido responsabilidad, esa que han tenido, esa que hemos tenido todos los riojanos y riojanas para llegar a donde hemos llegado. El día de la celebración, por favor, pido responsabilidad: mascarilla obligatoria, distancia y cuidado.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=e9E9Z0sBz1A&t=1s'), 'texto': 'Con esta cesión gratuita de 10 viviendas del parque público del Gobierno de La Rioja que gestiona el IRVI, materializamos una parte del trabajo del centro de coordinación para la respuesta a la crisis humanitaria en Ucrania. El IRVI, lo he dicho antes y es así, debe tener un enfoque social, y desde esa óptica nos hemos enfocado y hemos trabajado desde el principio. La vivienda es un derecho de la ciudadanía y es un elemento fundamental e indispensable para desarrollar una vida digna. En este sentido, además de los muebles, el instituto riojano de la vivienda ha invertido 55 mil euros en dar las altas necesarias tanto de luz, de agua, etcétera como del amueblamiento.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=C2Yf5NjtWN8'), 'texto': 'El pacto persigue varios objetivos fundamentales: por una parte, un acuerdo en cuanto a las medidas sanitarias que garanticen una serie de protocolos y condiciones para la vuelta a la educación presencial de una forma progresiva y, sobre todo, segura para todas las personas. Un segundo objetivo es garantizar la inclusión educativa en términos totales. La educación a distancia nos ha manifestado esta dificultad para que una buena parte de nuestro alumnado pueda seguir la actividad educativa a distancia, así que es otro objetivo inmediato. Y, finalmente, elaborar un modelo común de plan de contingencia en función de los distintos escenarios posibles a partir de ahora respecto a una posible normalidad, un confinamiento absoluto como hemos vivido estos dos meses, o periodos de transición variable entre la presencialidad y la educación a distancia.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=J6UxxIdNujg'), 'texto': 'Mi objetivo ha sido, es y seguirá siendo el de conseguir una sociedad justa. Ninguna política puede tener éxito si se olvida de las personas. Hoy comienza para La Rioja un nuevo tiempo, un tiempo para la igualdad de derechos, igualdad de oportunidades, igualdad entre hombres y mujeres. Un nuevo tiempo de esperanza y oportunidades para nuestros jóvenes. Un nuevo tiempo para lo público. Un nuevo tiempo para la sostenibilidad y la ecología. Un nuevo tiempo para todas esas cuestiones que tanto la sociedad como los compromisos internacionales demandan. Esto es sólo el principio, mi compromiso va mucho más allá.'})
videos.append({'video': YouTube('https://www.youtube.com/wath?v=wHyMcqpSH_U'), 'texto': 'Hoy queremos agradeceros muy especialmente el papel que habéis jugado en la profunda transformación que ha vivido nuestra comunidad. Y en este sentido quiero aprovechar este acto también para reconocer la tarea no sólo de los que hoy llegáis a vuestra jubilación sino de todos los empleados y las empleadas públicos en el sostenimiento de una democracia real y efectiva. Vuestro trabajo garantiza los derechos civiles, los derechos políticos y los derechos sociales y hace posible que valores fundamentales de nuestra sociedad como la igualdad, la justicia social y la libertad se materialicen en políticas y servicios públicos que mejoran cada día las condiciones de vida de riojanos y riojanas. El servicio público, cuando se empiezan a ver los resultados, cuando se traduce en ayuda a los ciudadanos, o en procesos terminados ya, ese, ese es el trabajo que vosotros habéis comprobado, es el más gratificante. Es el más gratificante que podemos encontrar. Lo habéis vivido durante decenas de años, y lo estamos pudiendo comprobar desde hace meses todas las personas que formamos parte del nuevo gobierno.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=Whg7Z76fpTk'), 'texto': 'Desde el Gobierno de La Rioja se quiere apoyar también al sector comercial. Se ha estado apoyando durante toda la pandemia con más de 14 millones de euros para 4000 solicitudes de comerciantes que han, bueno pues lo han pasado mal porque han tenido que cerrar sus negocios. Hoy es un día de alegría porque podemos volver más o menos a la normalidad, y ya lo han dicho: hay que apoyar el comercio de cercanía, esa atención personalizada tan tan buena que se hace aquí en La Rioja a todos los comerciantes, y estos tres días van a ser días de eso, de aprovechar. Animo no sólo a riojanos y riojanas sino a todos nuestros vecinos que nos estén oyendo, que se pasen por aquí para que comprueben la calidad del comercio de cercanía.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=RczUSZh-_xc'), 'texto': 'La guardería se hizo para tener dos aulas de bebés, dos de un año y dos de dos a tres. Entonces ahora mismo en funcionamiento tenemos una de bebés, un aula mixta, una de un año a dos y otra de dos a tres. La verdad es que estoy orgullosa. Es un proyecto que viene anterior, por supuesto a mí me ha tocado lo más bonito que es inaugurarlo, pero bueno, doy la enhorabuena también a todos los que lo proyectaron anteriormente y cómo se ha terminado consiguiendo. Los niños que se crían, que juegan y que comparten ratos con sus compañeros en un patio abierto como éste, con un horizonte tan bonito y con la naturaleza, esto se les queda dentro y van a ser unas personas mucho mejores, estamos seguros todos. Por lo tanto, no puedo estar más que orgullosa y doy la enhorabuena al alcalde y a todos los que procuraron que esto saliera adelante.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=lPPTXq1csj4'), 'texto': 'El ambiente de trabajo es fabuloso, el que se observa y, además, se ha comprobado, como bien nos han dicho, o sea enhorabuena por todas las partes. Y muchas gracias por quedaros en La Rioja. Contad con el apoyo, con todo el apoyo del gobierno para, no sólo para que lo que habéis hecho, sino si estáis pensando en algo innovador que podría, en el que podría ayudaros el gobierno, contad con nosotros porque es este camino o ninguno.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=LbP0ZAgtz2Y'), 'texto': 'Ollauri, este pueblo tan precioso de La Rioja Alta, abría su escuela después de 20 años cerrada. Han venido 11 niños de distintas edades, el mayor de 9 años, y hoy empiezan su nueva andadura de colegio en su propio en su propio pueblo. Es una oportunidad realmente. Y hay que decir que tanto las escuelas como los institutos son los lugares más seguros donde ahora los niños y las niñas se pueden, bueno, se pueden relacionar entre ellos y, además, de aprender las materias, volver otra vez a ese cariño y a relacionarse. La verdad es que es hoy un día de alegría. La escuela es lo que da vida al pueblo, lo que hace que la gente se quede a gusto, no tiene que desplazarse los niños… son muchas las ventajas que tiene y, sobre todo, que me parece que es clave para el desarrollo del pueblo el contar con la escuela aquí, que se queden los niños aquí.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=f_Otzzl-Z6c'), 'texto': 'Históricamente los momentos de crisis, de pestes, de enfermedades que se extienden son momentos grises, y si no hacemos por seguir cultivando la cultura que nos alimenta el espíritu y nos permite ser críticos y reflexivos pues no llegaremos a ningún sitio. Por lo tanto, de nuevo agradecimiento por esta iniciativa y que no decaiga.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=wnjkIsUxdJo'), 'texto': 'En defensa del interés general, el Gobierno de La Rioja ayer decidió iniciar el proceso de lesividad para dar un plazo de audiencia a los ayuntamientos, ayuntamientos que se vieron perjudicados por una nefasta gestión de 24 años, a los cuales les dieron la puntilla en este mes y medio de locura de verano del 19 en el que la consejería de fomento decidió pagar favores y servicios prestados dentro de su partido a costa de los impuestos de todos los riojanos. Le aseguro una cosa: el Gobierno de La Rioja no va a tolerar esa actitud. Vamos a cambiar de arriba abajo la financiación local para que sea una financiación justa, coherente y que colabore en la estrategia global de desarrollo de esta comunidad en vez de ser sectaria, arbitraria y clientelar, que es lo que ustedes promocionaron durante 4 años de gestión de política municipal.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=D-5_n--RWS0'), 'texto': 'Que estamos afectados por el tema de la pandemia, todas las brigadas de obras públicas nos estamos dedicando a hacer los trabajos urgentes de conservación de carreteras. Y al ser trabajadores esenciales nos estamos dedicando también a realizar desinfecciones por los pueblos de La Rioja Alta, principalmente, algunos pueblos de La Rioja Baja, también hemos hecho la señalización de los centros de que se hace de pruebas rápidas para la detección del coronavirus, también estamos llevando todas las empresas que hacen donaciones, llevándolas todas las donaciones al hospital San Pedro y al CIBIR y, principalmente, pues esas son todos los trabajos que estamos haciendo ahora en estos días.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=N-0sSD5Oyd4'), 'texto': 'A ver, nos ha cambiado mucho porque tenemos que estar en limpieza extrema y tenemos que tener mucho cuidado con todo, limpiar a fondo siempre y tener mucha responsabilidad con nuestro trabajo, que es lo primordial. Nuestro día a día de trabajo es con mucho cuidado y tenemos que protegernos también nosotras porque tenemos demasiado riesgo en este, en esta labor porque siempre, no sabemos en qué momento vamos a encontrar el COVID y tenemos que extremar todo, precauciones, limpieza… con todos nosotros y con todos los que están en nuestro área. Claro, con mucha emoción y muchas ganas de seguir adelante y seguir con nuestro laboral a tope. Muy conscientes de la labor que tenemos que hacer y con mucha responsabilidad.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=BEKe0-wHyX0'), 'texto': 'Todo el mundo venga aquí a La Rioja que los riojanos de todos los sectores vengan a colaborar, desde los hoteleros, los responsables de museos, ayer estaba la representante del museo Moore, todos todos los sectores, los turísticos de dentro y los de fuera, los internacionales, el turismo de la lengua. Tenemos tanto patrimonio en la lengua que lo que hemos hecho es establecer contactos al más alto nivel, que poco a poco irán dando los frutos. Y por último y más significativo por su nombre, que es La Rioja, hacer de La Rioja la eno-región. Hay muchísimo que hacer y tenemos a todo el mundo emocionado en eso, en poder llevarlo a cabo y poder desarrollarlo a lo grande. Me preguntaban: “pero, ¿y qué hay de nuevo?”. El patrimonio está, ese no es nuevo. Hay un nuevo gobierno con una estrategia muy potente basada en datos, y es la que vamos a llevar adelante.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=UL_r_kfPfiA'), 'texto': 'He querido entregar a los portavoces de los grupos parlamentarios que participan, que son todos, que participan en la comisión de reconstrucción social y económica que está trabajando desde hace semanas aquí en el parlamento. Les he querido entregar el documento donde se reflejan todas las cláusulas del acuerdo de reconstrucción social y económica que a su vez ya se firmó por parte del consejo de diálogo social y el gobierno. Son conclusiones que, bueno, que van a poder trabajar en esta semana y pico que queda hasta la comisión y final pleno donde se apruebe todo lo que lleven a esta comisión de trabajo. La conclusión final que sacamos todos es que queremos echar una mano todos los grupos parlamentarios, el consejo de diálogo social y el propio gobierno, todos a una, para mejorar esta situación económica y que, yo creo, sinceramente, creo que todos estamos trabajando por ello y que saldrá que será positivo al final.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=gly45xHxl4o'), 'texto': 'Es un día especial. Es muy especial para los vecinos de Arnedo, para los vecinos de Quel y es un día especial, si me permite, tengo el honor, para el Gobierno de La Rioja y quiero agradecer de aquí el trabajo que previamente hizo el Gobierno de La Rioja con su consejero a la cabeza. No hay que explicarle a nadie, riojano o riojana, que la agricultura para esta comunidad es y va a ser un pilar fundamental de nuestra economía, no la economía macro, que también, de nuestra economía riojana a nivel global, sino también de la economía familiar.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=OkBeeA6CLJ8&t=4s'), 'texto': 'El plan de vacunación frente al COVID-19 avanza a buen ritmo en La Rioja. Esta semana hemos abordado un importante número de ciudadanos de entre 55 y 59 años, y será también ese nuestro objetivo durante la próxima semana para lograr así completar pronto a este grupo y comenzar con la vacunación de las personas entre 50 y 54 años de toda la región. Si, como está previsto, aumentamos de manera sustancial nuestro ritmo de vacunación a partir de la semana que viene, La Rioja habrá logrado la inmunidad de grupo cuando esperamos, este mismo verano. Eso sí, para alcanzar esta meta necesitamos que los ciudadanos continúen diciendo sí a ponerse la vacuna. Hasta ahora lo están haciendo el 99% de los riojanos. La vacunación en menores de 60 años con AstraZeneca sigue en pausa. Según la decisión acordada esta misma semana, cuando termine esa pausa se administrará preferentemente la vacuna de Pfizer como segunda dosis. Esta vacunación será en tiempo y forma como siempre se ha hecho.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=CbqCR9j0wHc'), 'texto': 'Esta próxima semana continuaremos con la administración de la segunda dosis a personas de entre 70 y 79 años. Además, gracias al aumento en el volumen de vacunas recibidos, daremos un impulso a la vacunación en ciudadanos entre 60 y 69 años. El proceso de vacunación en La Rioja avanza a un buen ritmo: seguimos alcanzando nuevos hitos. Ya son más las personas inmunizadas que las contagiadas desde el inicio de la pandemia. Además, La Rioja es la comunidad donde hay un mayor número de personas mayores de 60 años que al menos hayan recibido una dosis. Ante la vacunación lo importante es destacar que las vacunas son seguras y son eficaces. La mejor vacuna es la vacuna que está administrada. Por lo tanto, lo importante es estar vacunado y contar con un grado de protección frente al virus No hay que tener miedo a ninguna de las vacunas, hay que tener miedo a infectarse por el virus.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=6FJBZRDDStU'), 'texto': 'El plan de vacunación frente al COVID-19 de La Rioja continúa avanzando a buen ritmo. De hecho, la próxima semana alcanzaremos un nuevo hito. Comenzaremos con la inmunización de manera programada a los primeros ciudadanos menores de 30 años. En concreto, administraremos las primeras dosis de la vacuna a la población de 29 años en algunas de las zonas básicas de salud de La Rioja Alta. Durante los próximos días la planificación continuará centrada sobre todo en inmunizar a la población de entre 30 a 49 años de toda la comunidad. Al tiempo que administraremos las primeras dosis a las generaciones de menor edad comprendidas en estos grupos, de manera paralela, seguiremos completando también la pauta de aquellos ciudadanos a quienes les corresponda recibir su segunda dosis. Debemos felicitarnos porque entre todos estamos logrando que, semana a semana, La Rioja siga siendo una de las comunidades con mejor ritmo de vacunación. Alcanzar la inmunidad de grupo y cumplir con las medidas anti covid hará posible que juntos logremos detener al virus.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=-kBKHpeSzlQ'), 'texto': 'Este gobierno, progresista y de izquierdas, insisto, apuesta por lo público, por fortalecer los servicios públicos, y lo hacemos con responsabilidad, cumpliendo la legalidad y siendo eficientes. Y para seguir protegiendo y ayudando a la ciudadanía este gobierno va a donar todo el material de protección que se ha suministrado durante estas semanas a las residencias de mayores, centros de menores y centros de discapacidad, ya sean públicos o privados, con un importe de más de 500 mil euros. Y, además, se donará todo el material de protección que ha ido suministrando el Gobierno de La Rioja a los ayuntamientos, desde el ayuntamiento de Villarroya, con cinco habitantes, hasta el ayuntamiento de Logroño, con 151 mil habitantes. Máscaras para adultos, máscaras para niños, pantallas de protección para niños, hidrogeles, todo ello por un importe de más de un millón y medio de euros. Y además de presupuesto, medios y personal, es necesario un reconocimiento. Y por eso he trasladado una propuesta para que el consejo de gobierno conceda la distinción de medalla de La Rioja al colectivo de trabajadores y trabajadoras sanitarias, socios sanitarios de La Rioja.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=jYZ-aQYCQKo'), 'texto': 'Mi impresión es que lo que hay detrás de vuestro trabajo y de vuestro esfuerzo es un compromiso. Y no es solo un compromiso con vosotros mismos, es un compromiso con la sociedad porque mejorando en vuestros ámbitos competenciales, evaluando vuestro trabajo, analizándolo con capacidad crítica y cambiando lo que no funciona mejoráis vosotros, pero también mejora la sociedad en su conjunto.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=w7EeTIrkIm0'), 'texto': 'Desde el Gobierno de La Rioja, como hemos manifestado en muchas ocasiones, hemos decidido propiciar la movilidad sostenible, entre ellas la movilidad ciclista con bicicleta de otras formas sostenibles y limpias de desplazarnos, todo porque nos parece que es, bueno, un cambio de paradigma necesario en pro de la salud de las personas, porque es un medio no contaminante que repercute en el ejercicio físico y en la salud mental y física de las personas, y también que reparte de forma ajustada el espacio en nuestras ciudades. Por eso hemos juntado hoy a todos los actores de la movilidad sostenible de La Rioja para presentarles un documento de trabajo que será el inicio de, pues, diferentes reuniones que tendremos, para mejorarlo y elaborar con ello la estrategia de la bicicleta de La Rioja. La primera estrategia de este tipo, que como digo incide en esta movilidad tan necesaria en nuestra tierra.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=0dWiqJDWexo'), 'texto': 'Por fin, bueno, hacemos lo que teníamos que hacer, que es poner en marcha este centro de salud en la zona de la Villanueva. La arquitecto nos ha explicado perfectamente los detalles del centro de salud, por lo tanto no quiero repetirme. Lo que sí quiero destacar es que es tres veces más grande, que tendrá quince consultas tanto de enfermería como consultas médicas, dos polivalentes nos ha comentado, que los materiales serán absolutamente, por supuesto, lavables, desinfectables, etcétera, y que es el primer centro de salud post pandemia. Es decir, adaptado a posibles, para hacer el frente a posibles contagios de virus, de cualquier tipo de virus, que impliquen separación clara dentro del centro de salud para las personas con riesgo de contactos o contagiadas y las personas. Por tanto, estamos mirando al futuro, y además estamos utilizando materiales, bueno, y la energía la geotermia nos ha comentado y fotovoltaica, es decir, energías renovables porque ese es el camino que tenemos que ir siguiendo para alcanzar esos objetivos del consumo mínimo de energías no renovables. Estamos, el gobierno de La Rioja está feliz de poder llevar a cabo esta esta obra por fin.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=oFfZtwlj8qc'), 'texto': 'Estamos dentro de lo que es una marca de hoja, una marca de calidad que es, lo que desde el Gobierno de La Rioja queremos potenciar, ¿no? La Rioja se distingue por su calidad, de sus productos, y uno de esos productos precisamente ese es el aceite. Por eso, animaros a todos los productores y a todos los que comercializáis, pues, esta joya que tenemos en La Rioja, que es el aceite, y poco a poco también nos van conociendo por el aceite.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=9dJP9XMBUTA&t=1s'), 'texto': 'Estamos en el parque de La Grajera de Logroño, un entorno de singular valor natural, en el que vamos a realizar la suelta de 43 aves y mamíferos de los que hemos ido recuperando tanto en el Centro de Recuperación de especies Silvestres de “La Fombera” como a través del programa de voluntariado social ecosocial “Ayúdale a volar”. El programa “Ayúdale a volar” ha podido rescatar durante este periodo, y todavía sigue en funcionamiento desde abril hasta septiembre, un total de 250 polluelos de diferentes especies de aves que han estado en familias y en centros educativos, en residencias de personas mayores. Han estado todas estas personas, estos voluntarios, recuperándoles, alimentándoles, cuidándoles. Lo que es una de las iniciativas pues más interesantes que tenemos de educación ambiental, que además esperemos que se replique a las familias de estas personas, a los amigos y que, que corra la voz y que cada vez más gente sea y participe en este en este voluntariado “Ayude a volar” que promociona y impulsa al Gobierno de La Rioja.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=Mm7FNrqCYaQ&t=1s'), 'texto': 'Es importante reflejar la sensibilidad sobre una realidad que forma parte de nuestra sociedad como es la discapacidad, y a través de esta actividad, bueno, pues ven esa situación. Entendemos que no sólo es una realidad que puede interesar a niños o a niñas más pequeños o más pequeñas, y que, pues, en definitiva, yo creo que el interés que manifiestan, pues, también los alumnos mayores es un es un interés que tenemos que apoyar. Por parte del centro, por parte del instituto hay una situación que es positiva en cuanto a conocimiento y al propio hecho de poder contrastar situaciones que se producen y nuestro interés, lógicamente, es la ampliación del servicio de lo que es la actividad en años próximos.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=oen6_8YQzo0'), 'texto': 'La realidad es que desde la consejería de Servicios Sociales y a la Ciudadanía trabajamos en este programa y lo vamos a seguir haciendo porque nos parece clave sensibilizar a la ciudadanía sobre el tema de la discapacidad y comenzar con los menores, aunque la realidad es que ellos no tienen ningún tipo de prejuicios. Los prejuicios los tenemos los mayores, ellos se integran muy bien y van a ser la vía que nos puedan enseñar a nosotros.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=uREyueqin6I'), 'texto': 'Creemos que una coordinación rápida e intensa entre el gobierno de la capital, el gobierno de la capital y el Gobierno de La Rioja es básica para poder dar aliento, como bien decía la presidenta, a muchos de los temas que están encima de la mesa. Para ver exactamente qué se puede mejorar de ese convenio de capitalidad necesario y que el Gobierno de La Rioja, por supuesto, está dispuesto a seguir con él y a mejorarlo, pero esa comisión de seguimiento como casi cualquier, en cualquier otro convenio son necesarias, ese seguimiento para saber qué se está haciendo y que se puede mejorar.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=s_3cvk1hQbs'), 'texto': 'Hemos mantenido una reunión con las organizaciones profesionales agrarias, la presidenta del gobierno Concha Andreu, los consejeros de hacienda, sostenibilidad y yo misma, como titular de agricultura. En esta reunión hemos abordado una serie de propuestas que nos plantean las organizaciones profesionales agrarias en relación con medidas fiscales. En la reunión se han analizado todas y cada una de ellas. Hay algunas de ellas que son de competencia estatal y hemos asumido el compromiso de trasladarlas a Madrid, y las que son de competencia autonómica, algunas de ellas ya se han recogido en el anteproyecto de la futura ley de agricultura y ganadería, y que está todavía en fase de exposición pública. Pero estas medidas autonómicas, de carácter autonómico, aquellas que no están recogidas en el anteproyecto de ley de agricultura y ganadería, nos hemos comprometido con las organizaciones profesionales agrarias en estudiarlas y valorarlas, siempre desde la colaboración con el sector y siempre desde la intención de este gobierno de apoyar la competitividad y la rentabilidad de un sector importante y estratégico en nuestra comunidad autónoma.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=7nkwYo4RwuM'), 'texto': 'El gobierno tiene que estar ayudándoos y escuchándoos, intentando solucionar todas las cuestiones que día a día se van planteando, que son realmente nuevas porque es una situación inédita de todo punto. Agradeciéndoos el esfuerzo, de antemano el que vais a hacer, pero ya el que habéis realizado hasta llegar hasta aquí y pidiéndoos cuatro cosas: precaución, serenidad, responsabilidad y colaboración. Estamos juntos en esto, y tanto mi equipo de gobierno con la consejería de educación a la cabeza, como yo misma vamos a estar a vuestra entera disposición. El reto que tenemos por delante es grande, cierto, pero lo es mucho más nuestro compromiso con el futuro de esta región.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=lET9xKucDTU'), 'texto': 'Y así, en conjunto y escuchándonos todos y con un diálogo permanente constructivo es, va a ser la clave de toda la tarea que tenemos por delante para corregir esta deriva económica hacia la baja y esta deriva de producción industrial hacia la baja que llevamos ya acumulando varios años. Hablar de desarrollo social, de políticas públicas, de apostar por un incremento de los salarios en nuestra región, de hablar de acabar y luchar contra las desigualdades pues evidentemente para comisiones obreras es una buena sintonía y un buen inicio. Es vital el mantenimiento y la potenciación de este espacio, el diálogo social, pues que ha permitido pues avanzar en algunas cuestiones que tienen que ver pues sobre todo con la creación de empleo estable de calidad, con aprovechar las oportunidades magníficas que tiene nuestra región.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=_ARoFX9FKjU'), 'texto': 'Para este camino es esencial el papel de todos y cada uno de los alcaldes, que son los que están al pie de la calle, de los problemas y de las soluciones también que ofrece cada uno de los ciudadanos. En las reuniones que he mantenido a lo largo de ya once domingos con el presidente del gobierno Pedro Sánchez y el resto de presidentes de comunidades y ciudades autónomas hemos dado una especial importancia, y más en esta última fase ya de más desescalada, la importancia a los alcaldes y alcaldesas de toda España. El resumen es realmente positivo. Estas reuniones se deben de seguir manteniendo porque son claves, nos dan pulso de la realidad, que es a lo que este gobierno y esta presidenta tiene que prestar atención, a la realidad de cada municipio.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=HP4jEc-osWg'), 'texto': 'Bueno, pues muy buenos días a todos y todas y muchas gracias, como siempre, por su asistencia a esta rueda de prensa posterior al consejo de gobierno. Hoy, como ven, tengo el placer de estar acompañado por el consejero de desarrollo autonómico José Ángel Lacalzada, quien nos va a compartir detalles sobre un nuevo proyecto económico que va a favorecer, y mucho, el desarrollo de La Rioja. Pero, con vuestro permiso, antes de nada, comenzaré dando cuenta de los temas abordados en el consejo de gobierno de hoy. Ya estamos terminando julio así que vamos a empezar con noticias festivas porque el consejo de gobierno ha aprobado el calendario de los 12 festivos laborales para el año 2023. Los festivos laborales para el 23 serán los siguientes: el viernes 6 de enero para hacerlo para festejar la costa 33 por navidad.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=nMlhZ6uVDxI'), 'texto': 'Toda la actuación de mi gobierno debe pues moverse dentro de los márgenes generosos que marca la ley. Dentro de la ley todo, fuera de la ley nada. Pero no siempre esos márgenes son claros, y es ahí donde juega su importante papel en nuestro sistema autonómico el consejo consultivo. Importancia tal que el propio estatuto de autonomía, ley orgánica estatal, garantía de nuestro autogobierno, lo eleva a la categoría de órgano superior consultivo cuya composición y funciones se regirán por los principios de imparcialidad e independencia. Estos principios, en la composición del órgano y en el desarrollo de sus funciones, que son de tanta ayuda a quienes de ellas nos benefician, son la garantía de que el criterio jurídico que manifestáis colegiadamente quede libre de la influencia de intereses espurios.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=At_kHhe62Kc&t=3s'), 'texto': 'En el mundo de la… bueno, de de la industria agroalimentaria, como es el del viñedo. Aquí, afortunadamente, hay una mayoría de mujeres en el equipo, y digo afortunadamente con conocimiento de causa, no por casualidad. Y veo que, bueno, que la organización, el estilo de vino y el estilo de la comercialización de esta bodega, pues, si me permitís, ha ido incrementando. Es importante seguir trabajando en el mundo de la cooperativa. Las cooperativas lo que hacen es fijar en el territorio a la población, y el mundo rural necesita de fijación de población. Estas cooperativas lo que hacen no sólo producir, bueno, llegar a producir calidad, sino también fijar población.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=yyqnFEL7hPM'), 'texto': 'Hemos conseguido en un tiempo récord superar y preparar el centro para que pasara la revisión de calidad que nos hace el organismo competente y, por tanto, pueden empezar a funcionar y poder empezar a trabajar cuanto antes en los temas de formación profesional. Va a ser una referencia para todos los ciclos formativos que actualmente se hacen en la Comunidad Autónoma de La Rioja para la enseñanza a distancia, lo cual va a multiplicar y mucho los número de alumnado que está presente en estos centros.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=guCTXQ1d2TY'), 'texto': 'Se trata de dotar de medios a la justicia, y eso es lo que hemos, durante, durante años hemos oído los políticos y es una demanda que teníamos que cumplir: dar medios a la justicia. De momento tenemos este juzgado, la verdad es que hoy es un día para estar contentos y de enhorabuena. El compromiso del Gobierno de La Rioja con la justicia, que es un servicio a la ciudadanía importante, es un compromiso firme. Somos muy conscientes de que hay, había necesidades reales de la creación de este juzgado porque el juzgado de lo penal número 1 y número 2 tienen cada uno 500 causas y 1000 ejecutorias, cada uno, para resolver, lo cual produce un atasco en detrimento de esa justicia a la que los ciudadanos tienen derecho.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=AnjJYJo4sF4'), 'texto': 'Solamente de la colaboración entre administraciones, como es el caso, salen adelante obras grandes, grandes proyectos. Esta obra forma parte del plan de integración urbana del ferrocarril de Logroño, de la ciudad de Logroño. Son 14 millones y medio, de los cuales el 55% lo acoge, por así decir, Logroño, y La Rioja, el Gobierno de La Rioja y el ayuntamiento. Se trata de una obra, lógicamente, para poner las infraestructuras riojanas y logroñesas a nivel de futuro.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=yHEF5f3RkO4&t=1s'), 'texto': 'Insistir en el mensaje de la colaboración entre administraciones. De esta manera, y teniendo todos claro hacia dónde vamos y qué es lo que queremos con nuestro patrimonio, que al fin y al cabo es el que nos da la identidad y nos da los valores para seguir transmitiéndoselo a las generaciones que vendrán después. Briones tiene además la suerte de tener una alcaldesa que es experta, que es doctora, en historia, correcto. Y valora, en sus términos justos, además del gobierno de España por supuesto, y el propio y la propia consejería del gobierno, valora en sus propios términos el valor tan fundamental que tiene conservar nuestro patrimonio. Además, Briones va a ser parte fundamental de esa mancomunidad. Briones, Ábalos, San Vicente, San Asensio, que están trabajando para el futuro del turismo enológico, que va a ser clave en el proyecto de no región también. Por lo tanto, estamos de enhorabuena hoy. Conservamos patrimonio para potenciar nuestro turismo en la economía al fin y al cabo y proteger esa identidad que tenemos que pasarles a nuestras generaciones futuras.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=3cPgXNmxU2o'), 'texto': 'En la situación social en la que vivimos, estas sociedades avanzadas con tanto dinero en determinadas personas y tan poco en otras, lo que tendríamos que hacer es, por encima de todo, trabajar la justicia social. Por una parte, admiro el trabajo que están haciendo, pero por otra hay que reivindicar que esto debería desaparecer porque tendría que estar mejor repartida, mejor repartida la riqueza. Eso es tarea de los que gobernamos, y esa es una de las tareas que yo me propongo. No que desaparezca, todo lo contrario, sino de que no haya que hacer este esfuerzo de solidaridad de todos los ciudadanos para atender a las personas que más lo necesitan.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=l5_1XkXf6DE'), 'texto': 'En Logroño tenemos cuatro centros de participación activa, que cada uno de ellos tiene un centro de día con 20 plazas. Y en La Rioja, distribuidas por toda la comunidad, tenemos 12 centros de participación activa. Hay dos objetivos muy importantes en esta consejería, que uno es el envejecimiento activo, es decir, que las personas mayores puedan envejecer participando lo más posible en la sociedad y con un buen nivel de calidad de vida, y por otra el de las personas que están en el centro de día y que requieren de cuidados para retrasar la aparición de la dependencia.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=duUqD3gu0Ak'), 'texto': 'Hemos querido venir a visitar este nuevo piso para la Universidad Popular de Logroño, la UPL. Un piso que estaba, bueno, propiedad de la comunidad autónoma de La Rioja, pero que estaba sin utilizar desde hace algún año, y afortunadamente hemos podido trabajar conjuntamente con la UPL y hoy es el día en el que venimos a comprobar que, efectivamente, están están utilizándolo y están desde las nueve y media de la mañana o 9 de la mañana hasta las 10 de la noche utilizándose. Por lo tanto, estamos contentos. La UNESCO lo tiene claro: la educación debe darse en todas las fases de la vida. Y qué mejor que en la última laboral, es decir, en cuanto terminas de trabajar, bueno, poder tener una universidad que te acoge, que te enseña y que te prepara para la relajación como hemos visto aquí una de las clases. Por lo tanto, estamos orgullosos de poder dar uso al patrimonio que tiene la Comunidad Autónoma de La Rioja.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=p7z0JtHNT4s'), 'texto': 'Quiero decir que estoy muy, muy orgullosa del equipo tanto de emergencias de enfermeras, enfermeros, la cruz roja que colabora también con el gobierno para que esto salga bien de manera fluida, para que ninguna persona que tiene que ser vacunada se quede sin vacunar. Es un hito y lo estamos haciendo, quiero decir que lo están haciendo, muy bien. Estamos en, dentro de los puestos de todo el país que más vacunas estamos poniendo a la semana. Así que, bueno, pues tenemos que alcanzar lo que queremos alcanzar, que es esa inmunidad de grupo a lo largo del verano y vamos por muy buen camino.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=sRhuHFf3Wj0'), 'texto': 'Nos hemos trasladado aquí para ver la transformación de esta empresa. Han conseguido con el esfuerzo, el esfuerzo solidario de todos y cada uno de los trabajadores y de los propios hermanos, que son los dueños de la empresa, han conseguido idear material de protección absolutamente innovador. Me lo transmitía ahora mismo la familia. Ha sido el impulso, la ilusión de todas las personas que trabajan aquí, de todo el pueblo de Arnedo y del Gobierno de La Rioja, que ha estado codo con codo, de verdad, echando una mano, viendo a ver cómo se podía buscar solución a los problemas que había.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=6VZHo3VbFJ8'), 'texto': 'Acabamos de contratar a 18 personas que van a trabajar aquí tanto médicos, doctores, enfermeras, enfermeros, celadores. ¿Por qué? Porque no solamente son instalaciones nuevas, aisladas por tanto del circuito general, sino que además cuentan con profesionales especialistas en pediatría. Todo ello enfocado a mejorar la calidad asistencial, la calidad del servicio sanitario riojano que tanto apreciamos y este gobierno tanto quiere fortalecer a pesar de la situación compleja de pandemia.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=xMTO0EOcm5Q'), 'texto': 'Estamos en Berceo, en pleno valle del río Cárdenas, en pleno valle de la Lengua, visitando las obras de la travesía que la Dirección General de Infraestructuras de la consejería ha realizado. Una inversión de casi 300.000€ en una distancia de 600 metros de travesía que va a mejorar la seguridad vial de las personas del valle, y también va a facilitar las visitas a un centro de cultural como es el Monasterio de San Millán de la Cogolla, los monasterios de Suso y Yuso. Es una apuesta del Gobierno de La Rioja por las zonas rurales, por luchar contra el reto demográfico y también por facilitar este turismo que tiene que vertebrar la zona y tiene que ser la base de la actividad socioeconómica, un turismo sostenible y responsable en esta parte de La Rioja que nos da la identidad y nos da personalidad.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=bgTHfQU_Uac'), 'texto': 'El Gobierno de La Rioja al que represento tiene que llevar adelante esa pasión, esa pasión por la solidaridad con los que más lo necesitan y, por lo tanto, con actos tan con honrosos como como éste.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=9dJP9XMBUTA&t=1s'), 'texto': 'Estamos en el parque de La Grajera de Logroño, un entorno de singular valor natural, en el que vamos a realizar la suelta de 43 aves y mamíferos de los que hemos ido recuperando tanto en el Centro de Recuperación de especies Silvestres de “La Fombera” como a través del programa de voluntariado social ecosocial “Ayúdale a volar”. El programa “Ayúdale a volar” ha podido rescatar durante este periodo, y todavía sigue en funcionamiento desde abril hasta septiembre, un total de 250 polluelos de diferentes especies de aves que han estado en familias y en centros educativos, en residencias de personas mayores. Han estado todas estas personas, estos voluntarios, recuperándoles, alimentándoles, cuidándoles. Lo que es una de las iniciativas pues más interesantes que tenemos de educación ambiental, que además esperemos que se replique a las familias de estas personas, a los amigos y que, que corra la voz y que cada vez más gente sea y participe en este en este voluntariado “Ayude a volar” que promociona y impulsa al Gobierno de La Rioja.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=GHOR7Ol1leE'), 'texto': 'Es claro que en esta vida tenemos que buscar la igualdad de oportunidades para todo el mundo, y en este caso más todavía, ¿no? Buscar esa plena inclusión, una plena inclusión que es un derecho para todas estas personas. Pero también desde la sociedad, desde las administraciones públicas, tenemos una responsabilidad muy grande en ayudarles en buscar esta plena inclusión. Por lo tanto, eso es lo que van a tratar hoy en estas jornadas y yo les animo a que los trabajos sean fructíferos y que mejor sitio que la Universidad de La Rioja, que es una universidad que está integrada dentro lo que es nuestra sociedad para desarrollar estas formas.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=WpLh9Dyw1I4'), 'texto': 'Una feria que se encuentra, digamos, en su 25 aniversario, que, como saben, es una cita referente en todo el norte de España. El mueble sigue siendo un sector clave para la economía de La Rioja con un peso importante en términos de generación de empleo, y en concreto en lo que representa para la comarca y para la ciudad de para la ciudad de Nájera. Con esta feria lo que pretendemos es promocionar el sector comercial del mueble y la Asociación del Mueble de Nájera como un factor de cohesión y de dinamización para este para este sector. Queremos también consolidar la imagen de Nájera como capital del mueble y dinamizar, por lo que supone de atractivo de gente, dinamizar el comercio de la localidad y de su área de influencia.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=eQhz2gJFNp8'), 'texto': 'La formación profesional hoy día yo creo que ha dejado de ser algo marginal, ¿no? y que es una formación profesional que da posibilidad a nuestros jóvenes de incorporarse a lo que es la inserción laboral. Y tenemos que decir, además, que en La Rioja la formación profesional, como se suele decir chuta, ¿no? que va que va bien, va bien porque tenemos más de 7.000 alumnos, tenemos como le decía 20 familias de formación profesional, 68 ciclos formativos.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=gYnEX-T4ou0'), 'texto': 'Creo que Europa nos ofrece hoy unas nuevas alternativas, unas nuevas soluciones ante los retos que tenemos económicos, sociales, de empleo, de vertebración, de lograr un equilibrio humano mucho más sostenible en nuestros municipios. Tenemos que recorrer este camino con la Unión Europea. Será un buen compañero de viaje, lo está siendo para la administración autonómica, lo puede estar también para nuestros municipios y desde luego os invito también a través de los grupos de acción local a ir sumando esa mayor financiación comunitaria en nuestras en nuestras poblaciones.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=ZMGxBQ7XsI4'), 'texto': 'Ese crecimiento económico se ha traducido en generación de empleo. Sé que a estos a usted les duele. Mire, por coger de forma interanual los últimos cuatro años, desde febrero del 2015 a febrero de 2019, más de 9000 riojanos y riojanas han salido o han abandonado las listas del paro, lo que representa un 36%. Y eso, señoría, se hace con crecimiento económico. Es decir, el crecimiento económico lo hemos sabido transformar mejor que otras comunidades autónomas en lo que es un gran desarrollo social. Y hemos situado a La Rioja como la comunidad de régimen común con mayor calidad de vida. La Rioja es la comunidad con mayor que mayor intensidad ha reducido la tasa de pobreza, la que más eficazmente ha combatido y ha reducido la tasa de fracaso escolar, señoría, y somos la comunidad con el menor grado de desigualdad social.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=edj9dqXdu_Y'), 'texto': 'Esa fortaleza común nos llevó, en el marco del parlamento autonómico, a desarrollar la ley de medidas de apoyo a las víctimas del terrorismo. Pero nos debe llevar también a preservar la memoria colectiva de lo sucedido, a no permitir bajo ningún concepto o bajo falsas premisas de convivencia que la historia se cuente de forma sesgada y tergiversada. La posición de las víctimas, como referentes éticos de nuestro sistema democrático y de derecho, está clara. También debe de estarlo el lugar que ocupan sus asesinos. Para ello es fundamental que el relato verdadero llegue a toda la sociedad, pero especialmente a los más jóvenes.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=vUtzv3PMjpc'), 'texto': 'Las obras, como les decía, importantes. Importantes por la cuantía, casi un millón doscientos mil euros, en concreto un millón ciento cincuenta mil euros de inversión, que es una hora importante. Yo creo que la de mayor inversión en lo que es adecuar parte urbana de la ciudad. Y, sobre todo, por el trazado en sí, ¿no? Yo creo que es una obra muy especial que se actúa en 6000 metros cuadrados del propio corazón de la ciudad de Alfaro en 11 calles.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=-mQTHW0uge8'), 'texto': 'Lo cierto es que aquí, en este parlamento, ha habido una mayoría que ha apostado por el progreso de nuestra tierra. Y yo lo quiero agradecer. Una mayoría política que ha permitido que La Rioja haya sido gobernada desde la responsabilidad y el acuerdo, aunque les pese a algunos, señorías. Y eso, no me cansaré de repetirlo, ha sido bueno para todos los ciudadanos de La Rioja. En realidad, la aprobación de esta proposición de ley pone el broche a una legislatura marcada por el diálogo, por el la escucha y por el acuerdo. Y, sobre todo, mirando a lo que es la gobernabilidad de La Rioja, que en todo momento ha sido el objetivo prioritario de este gobierno.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=ucx_iWKU0NU'), 'texto': 'Es una ley que deja indefensos y claramente desprotegidos colectivos como son los veterinarios, como son los dueños de animales o incluso a todos los ayuntamientos de La Rioja. Por eso el Gobierno de La Rioja tenía la obligación de presentar este recurso ante el constitucional. El Gobierno de La Rioja ha querido regular de forma sensata lo que es la protección de los animales. No mediante esta ley como han hecho de forma chapucera.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=acZe6-VlzRE'), 'texto': 'Las cosas se están haciendo bien en nuestra universidad. Y que podemos presumir de talento, de creatividad, de esfuerzo, de apuesta por la mejora permanente y la innovación. Y todo ello en beneficio del conocimiento y del papel insustituible de la universidad como motor del crecimiento, el desarrollo y la transformación de nuestra sociedad.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=I1YL5bUT-zM'), 'texto': 'Desde la administración, desde la consejería de educación, estamos en constante renovación y acondicionamiento de muchos centros de nuestra comunidad autónoma a lo largo de estos últimos años. En los últimos meses hemos inaugurado el colegio de Cenicero, el colegio de Medrano. En estos días estaremos también en el colegio de Albelda de Iregua. Se está trabajando ya en la ampliación del colegio de Villamediana, en el instituto de Rincón de Soto, de Pradejón. Bueno, yo creo que desde la consejería no paramos en acondicionar y en adecuar las instalaciones educativas, buscando esa educación de calidad con nuestra comunidad autónoma.'})
videos.append({'video': YouTube('https://www.youtube.com/watch?v=VcXzL2E8jXo'), 'texto': 'Con estas medidas queremos integrar el principio de igualdad en todas las políticas públicas incorporando nuevos modelos de gobernabilidad. Impulsar la igualdad real en la propia administración y desde la propia administración. Queremos avanzar también hacia un nuevo modelo de utilización del tiempo. Del tiempo en pro de la calidad de vida. Se trata de situar el tiempo de las personas en el centro de ese modelo. Las administraciones tenemos el compromiso de impulsar estrategias que generen nuevos modelos de horarios y de uso del tiempo dentro de lo que es el tejido socioeconómico.'})
print(len(videos))

74


In [3]:
for v in videos:
    v['video'].streams.filter(only_audio=True)
    audio = v['video'].streams.get_by_itag(22)
    v['mp4'] = audio.download()
    audio = AudioSegment.from_file(v['mp4'])
    audio.export('video.raw', format='raw')
    v['audio'] = audio

In [4]:
videos[0]['audio']

Como el modelo acepta audios únicamente de una duración limitada, separamos los audios en trozos de 60 segundos para que no de error. Además, los hacemos solapar durante 8 segundos para que no haya problema a la hora de dejar una palabra cortada por la mitad.

In [5]:
j=0
for v in videos:
    audio = v['audio']
    audio_length = len(audio) 
    print(f'Duración del audio: {audio_length}') # lo mide en milisegundos
# chunk counter 
    chunk_counter = 1
    
    point = 60000  # duración de cada trozo
    rem = 8000  # tiempo de solapación
    flag = 0
    start = 0
    end = 0

    lista ={}  # me creo un diccionario donde ir almacenando las rutas con sus chunk

    for i in range(0, 2 * audio_length, point):
        if i == 0:
            start = 0
            end = point
        else:
            start = end - rem
            end = start + point
        if end >= audio_length:
            end = audio_length
            flag = 1  # para que pare
        chunk = audio[start:end] # nos quedamos con la parte del audio

        chunk_name = f'chunk_{j}_{chunk_counter}' # le damos un nombre distinto a cada chunk

        chunk.export(chunk_name, format = 'raw') # guardamos el chunk
   
        lista[chunk_name+'.raw'] = chunk   # me guardo el chunk en la lista
        v['lista'] = lista
        print(f'{chunk_name} start: {start} end: {end}')
        chunk_counter += 1
        if flag == 1:
            break
    j = j+1

Duración del audio: 67733
chunk_0_1 start: 0 end: 60000
chunk_0_2 start: 52000 end: 67733
Duración del audio: 74931
chunk_1_1 start: 0 end: 60000
chunk_1_2 start: 52000 end: 74931
Duración del audio: 67175
chunk_2_1 start: 0 end: 60000
chunk_2_2 start: 52000 end: 67175
Duración del audio: 29861
chunk_3_1 start: 0 end: 29861
Duración del audio: 48901
chunk_4_1 start: 0 end: 48901
Duración del audio: 53174
chunk_5_1 start: 0 end: 53174
Duración del audio: 41401
chunk_6_1 start: 0 end: 41401
Duración del audio: 60140
chunk_7_1 start: 0 end: 60000
chunk_7_2 start: 52000 end: 60140
Duración del audio: 48181
chunk_8_1 start: 0 end: 48181
Duración del audio: 71053
chunk_9_1 start: 0 end: 60000
chunk_9_2 start: 52000 end: 71053
Duración del audio: 28166
chunk_10_1 start: 0 end: 28166
Duración del audio: 67570
chunk_11_1 start: 0 end: 60000
chunk_11_2 start: 52000 end: 67570
Duración del audio: 41842
chunk_12_1 start: 0 end: 41842
Duración del audio: 48576
chunk_13_1 start: 0 end: 48576
Duració

In [6]:
videos[0]['lista']['chunk_0_1.raw'] #Comprobamos que lo ha hecho bien

In [7]:
videos[0]['lista']

{'chunk_0_1.raw': <pydub.audio_segment.AudioSegment at 0x7f55fd5177d0>,
 'chunk_0_2.raw': <pydub.audio_segment.AudioSegment at 0x7f55fdbf05d0>}

Una vez tenemos todos los trozos separados y almacenados procedemos a transcribir cada uno de ellos y a guardarlos en otra lista.

Para ello, lo primero que hacemos es cargar el modelo que vamos a utilizar.

In [8]:
model_path = get_model_path()

config = {
    'verbose': False, 
    'hmm': os.path.join(model_path, 'es-es'),  #Carpeta del modelo acústico
    'lm': os.path.join(model_path, 'es-20k.lm.bin'), #Modelo de lenguaje
    'dict': os.path.join(model_path, 'es.dict')
}

In [9]:
j = 0
for v in videos:
    audio_paths = []
    transcriptions = []
    for i in range(1,len(v['lista'])+1):
        texto = ''
        audio_paths.append('chunk_'+str(j)+'_'+str(i))
        audio = AudioFile(**config, audio_file = audio_paths[i-1])
        for frase in audio:
            texto = texto + str(frase) +' '
        transcriptions.append(texto)
    v['path'] = audio_paths
    v['transcripcion'] = transcriptions
    j = j+1

In [10]:
print(videos[0]['path'])
print(videos[0]['transcripcion'])

['chunk_0_1', 'chunk_0_2']
["va a poder desde en edad de de dar den jackson los rumores de sr hill ok da en un producto red red sí compré uno un error de terror en el otro en la en el ar en él error rock ford gorda perder eran un error millón ven es red de un récord error sr ud donar un ar riesgo jill der error edad dudo ma m ma silla dos sr llegar lourdes llorar robado no rock producto en actas rick ud ven jack compré un duda duda guerras lleno un error un error t on mundos rick rick alrededor en en bajo error el hits de dornan en vez de productores de celda con una red bush actos redactar los llega en las rodillas en villa rebecca en un orden era tarde en el in josh sus rodillas del alrededor kurt kit del ira en el horno un como un acto de des rodar douglas robo es red um nick lou bajos en él y lionel alrededor ingrid se nos para elección en y en es red de millas pillo de dillard's uso verdad el tv ven ", 'un el anillo de dos millas ruda um ar verdad ud el de tv ven illa no un millón

In [11]:
print(videos[0]['texto'])

Defendamos esa bandera pionera contra la violencia de género. Pondremos en riesgo espacios de libertad y de igualdad alcanzados con mucho esfuerzo. El Gobierno de La Rioja ya lo ha hecho creando esta dirección general de igualdad que represento. Con su creación no sólo ofrecemos un mensaje claro sobre la esencia que internará toda la acción del gobierno, sino damos un evidente empuje a medidas dirigidas a acabar con la discriminación hacia las mujeres y a eliminar la violencia de género. Nuestra presencia en puestos de responsabilidad en tribunas como la que hoy utilizo para dirigirme a vosotros y a vosotras ya no son excepción. Avanzamos las mujeres, avanzáis los hombres, avanzamos como sociedad. En La Rioja ya son diez años los que llevamos sin lamentar víctimas mortales por violencia de género. Sí, hay razones para el optimismo, pero todavía nos queda mucho por hacer.


In [12]:
videos[0]['transcripcion']

["va a poder desde en edad de de dar den jackson los rumores de sr hill ok da en un producto red red sí compré uno un error de terror en el otro en la en el ar en él error rock ford gorda perder eran un error millón ven es red de un récord error sr ud donar un ar riesgo jill der error edad dudo ma m ma silla dos sr llegar lourdes llorar robado no rock producto en actas rick ud ven jack compré un duda duda guerras lleno un error un error t on mundos rick rick alrededor en en bajo error el hits de dornan en vez de productores de celda con una red bush actos redactar los llega en las rodillas en villa rebecca en un orden era tarde en el in josh sus rodillas del alrededor kurt kit del ira en el horno un como un acto de des rodar douglas robo es red um nick lou bajos en él y lionel alrededor ingrid se nos para elección en y en es red de millas pillo de dillard's uso verdad el tv ven ",
 'un el anillo de dos millas ruda um ar verdad ud el de tv ven illa no un millón en tennessee lou baja en 

Ahora que tenemos todos los trozos transcritos los concatenamos en una única variable que contenga todo el texto del vídeo transcrito.

In [13]:
for v in videos:
    texto = v['transcripcion'][0]
    for i in range(1,len(v['transcripcion'])):
        a = len(v['transcripcion'][i])
        j=1  #empiezo por 1 porque 0 será espacio
        esta = False
        long = len(texto)
        esp = texto[::-1].index(' ')
        texto = texto[:long-esp]    # corto a partir del último espacio por si la palabra está cortada
        esp2 = v['transcripcion'][i].index(' ') 
        v['transcripcion'][i] = v['transcripcion'][i][esp2:] # corto a partir del primer espacio por si la palabra está cortada
        while esta == False and j<a:
            if v['transcripcion'][i][1:j] in texto[:]:
                j = j+1
            else:
                texto = texto + v['transcripcion'][i][j:]
                esta = True
    v['transcripcion_unida'] = texto

Veamos la métrica que obtendríamos si no normalizáramos los textos.

In [15]:
media = 0
for v in videos:
    media = media + wer(v['texto'], v['transcripcion_unida'][0])
print(media/len(videos))  

0.9987380564683701


Ahora creamos una función que normalice el texto, pues una misma palabra puede estar escrita de varias formas diferentes.

In [16]:
def numero_to_letras(numero):
	indicador = [("",""),("mil","mil"),("millon","millones"),("mil","mil"),("billon","billones")]
	entero = int(numero)
	decimal = int(round((numero - entero)*100))
	#print 'decimal : ',decimal 
	contador = 0
	numero_letras = ""
	while entero >0:
		a = entero % 1000
		if contador == 0:
			en_letras = convierte_cifra(a,1).strip()
		else :
			en_letras = convierte_cifra(a,0).strip()
		if a==0:
			numero_letras = en_letras+" "+numero_letras
		elif a==1:
			if contador in (1,3):
				numero_letras = indicador[contador][0]+" "+numero_letras
			else:
				numero_letras = en_letras+" "+indicador[contador][0]+" "+numero_letras
		else:
			numero_letras = en_letras+" "+indicador[contador][1]+" "+numero_letras
		numero_letras = numero_letras.strip()
		contador = contador + 1
		entero = int(entero / 1000)
	
	if decimal == 0:
		numero_letras = numero_letras
	else:
		numero_letras = numero_letras+" con " + str(decimal) +"/100"
	#print('numero: ',numero)
	#print(numero_letras)
	return numero_letras
 
 
def convierte_cifra(numero,sw):
	lista_centana = ["",("cien","ciento"),"doscientos","trescientos","cuatrocientos","quinientos","seiscientos","setecientos","ochocientos","novecientos"]
	lista_decena = ["",("diez","once","doce","trece","catorce","quince","dieciseis","diecisiete","dieciocho","diecinueve"),
					("veinte","veinti"),("treinta","treinta y "),("cuarenta" , "cuarenta y "),
					("cincuenta" , "cincuenta y "),("sesenta" , "sesenta y "),
					("setenta" , "setenta y "),("ochenta" , "ochenta y "),
					("noventa" , "noventa y ")
				]
	lista_unidad = ["",("un" , "uno"),"dos","tres","cuatro","cinco","seis","siete","ocho","nueve"]
	centena = int (numero / 100)
	decena = int((numero -(centena * 100))/10)
	unidad = int(numero - (centena * 100 + decena * 10))
	#print "centena: ",centena, "decena: ",decena,'unidad: ',unidad
 
	texto_centena = ""
	texto_decena = ""
	texto_unidad = ""
 
	#Validad las centenas
	texto_centena = lista_centana[centena]
	if centena == 1:
		if (decena + unidad)!=0:
			texto_centena = texto_centena[1]
		else :
	 		texto_centena = texto_centena[0]
 
	#Valida las decenas
	texto_decena = lista_decena[decena]
	if decena == 1 :
		texto_decena = texto_decena[unidad]
	elif decena > 1 :
		if unidad != 0 :
			texto_decena = texto_decena[1]
		else:
			texto_decena = texto_decena[0]
 	#Validar las unidades
 	#print "texto_unidad: ",texto_unidad
	if decena != 1:
 		texto_unidad = lista_unidad[unidad]
 		if unidad == 1:
 			texto_unidad = texto_unidad[sw]
	return "%s %s %s" %(texto_centena,texto_decena,texto_unidad)

In [17]:
def normalizar(texto):
    #print(texto)
    tokens=word_tokenize(texto) #Separa el texto por palabras
    #print(tokens)
    texto = [w.lower() for w in tokens] #Pasamos las palabras a minúsculas
    #print(texto)
    texto = [texto[i].replace(' apdo.', ' apartado').replace(' art. ',' articulo ').replace(' atte.',' atentamente').replace(' avda.',' avenida').replace(' cap.',' capítulo').replace(' cía.',' compañía').replace(' coord.',' coordinador').replace(' d.',' don').replace(' dña.',' doña').replace(' dcho.',' derecho').replace(' dcha.',' derecha').replace(' depto.',' departamento').replace(' dr.',' doctor').replace(' dra.',' doctora').replace(' etc.',' etcétera').replace(' fdo.',' firmado').replace(' izqdo.',' izquierdo').replace(' izqda.',' izquierda').replace(' max.',' máximo').replace(' min.',' mínimo').replace(' núm.',' número').replace(' pág.',' página').replace(' ej.',' ejemplo').replace(' prov.',' provincia').replace(' sr.',' señor').replace(' sra.',' señora').replace(' srta.',' señorita').replace(' tfno.',' teléfono') for i in range(len(texto))] #quitamos las abreviaciones
    for i in range(len(texto)):
        if texto[i].isdigit() and (texto[i] not in string.punctuation):
            texto[i]=numero_to_letras(int(texto[i]))
            tokens2=word_tokenize(texto[i])
            for j in range(len(tokens2)-1):
                texto.append('a') #me añado esto simplemente para no tener problemas con el rango
            for j in range(i+1, len(texto)-3):
                texto[len(texto)-j+i]=texto[len(texto)-j+i-len(tokens2)+1]
            for j in range(i, i+len(tokens2)-1):
                texto[j+len(tokens2)-1] = texto[j]
            for j in range(i, i+len(tokens2)):
                texto[j]=tokens2[j-i]
    tokens=[w.lower() for w in texto] #Pasamos las palabras a minúsculas por si había números
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('',w) for w in tokens]
    stripped=[stripped[i].replace('¿', '').replace('¡','').replace("'",'') for i in range(len(tokens))] #quita los símbolos de puntuación que string.punctuation no tiene en cuenta
    words = [word for word in stripped if word.isalpha()] #Elimina los signos de puntuación
    return words

Una vez tenemos la función creada, normalizamos tanto el texto transcrito como el texto original.

In [18]:
for v in videos:
    v['texto_norm'] = normalizar(v['texto'])
    v['transcripcion_norm'] = normalizar(v['transcripcion_unida'])

Pasamos por un corrector de texto por si hay alguna falta ortográfica.

In [19]:
for v in videos:
    for word in v['texto_norm']:
        word = spell(word)
    for word in v['transcripcion_norm']:
        word = spell(word)

Para normalizar los textos hemos tenido que separarlo por palabras, así que volvemos a juntarlo todo en un único texto.

In [20]:
for v in videos:
    texto_norm = ""
    transcripcion_norm = ""
    for i in range(len(v['texto_norm'])):
        texto_norm = texto_norm + ' ' + v['texto_norm'][i]
    v['texto_norm'] = texto_norm
    for i in range(len(v['transcripcion_norm'])):
        transcripcion_norm = transcripcion_norm + ' ' + v['transcripcion_norm'][i]
    v['transcripcion_norm'] = transcripcion_norm

Y para acabar calculamos el error cometido por el modelo.

In [21]:
media = 0
for v in videos:
    media = media + wer(v['texto_norm'], v['transcripcion_norm'])
print(f'El modelo comete un error de {media/len(videos)}')

El modelo comete un error de 1.7087646692235876
